In [17]:

from torch import nn
from models.deeplabv3 import deeplabv3_mobilenetv2
from datasets.idda import IDDADataset
import utils.ss_transforms as sstr
import torch
from utils.utils import MeanReduction

In [15]:
model=deeplabv3_mobilenetv2(16)
model.load_state_dict(torch.load('round_28.pt',map_location=torch.device('cpu')))
iddaTestDiff=IDDADataset('data/idda',fileName='test_diff_dom.txt',transform=sstr.ToTensor(),client_name='test_diff_domain')
model.eval()
out=model(iddaTestDiff[0][0].unsqueeze(0))['out']

In [18]:
criterion = nn.CrossEntropyLoss(ignore_index=255, reduction='none')
reduction=MeanReduction()
l=reduction(criterion(out,iddaTestDiff[0][1].unsqueeze(0)),iddaTestDiff[0][1].unsqueeze(0))
print(l)

tensor(3.8691, grad_fn=<MeanBackward0>)


In [21]:
criterion = nn.CrossEntropyLoss(ignore_index=255, reduction='mean')
l=criterion(out,iddaTestDiff[0][1].unsqueeze(0))
print(l)

tensor(3.8691, grad_fn=<NllLoss2DBackward0>)


In [23]:
for i in iddaTestDiff[0][1].unsqueeze(0).unique():
            print(i)

tensor(0)
tensor(1)
tensor(2)
tensor(3)
tensor(4)
tensor(5)
tensor(6)
tensor(8)
tensor(10)
tensor(11)
tensor(13)
tensor(255)


In [40]:
class MeanReductionPerClass():
    def __call__(self, x, target):
        sum=0
        classes=target.unique()
        classes=classes[classes!=255]
        for i in classes:
            print(i)
            print(x[target==i].mean())
            sum+=x[target==i].mean()
        return sum/len(classes)

In [41]:
criterion = nn.CrossEntropyLoss(ignore_index=255, reduction='none')
reduction=MeanReductionPerClass()
l=reduction(criterion(out,iddaTestDiff[0][1].unsqueeze(0)),iddaTestDiff[0][1].unsqueeze(0))
print(l)

tensor(0)
tensor(1.8140, grad_fn=<MeanBackward0>)
tensor(1)
tensor(6.9221, grad_fn=<MeanBackward0>)
tensor(2)
tensor(3.4182, grad_fn=<MeanBackward0>)
tensor(3)
tensor(9.3305, grad_fn=<MeanBackward0>)
tensor(4)
tensor(0.3525, grad_fn=<MeanBackward0>)
tensor(5)
tensor(4.3332, grad_fn=<MeanBackward0>)
tensor(6)
tensor(3.3715, grad_fn=<MeanBackward0>)
tensor(8)
tensor(0.7109, grad_fn=<MeanBackward0>)
tensor(10)
tensor(7.6629, grad_fn=<MeanBackward0>)
tensor(11)
tensor(7.3871, grad_fn=<MeanBackward0>)
tensor(13)
tensor(4.9312, grad_fn=<MeanBackward0>)
tensor(4.5667, grad_fn=<DivBackward0>)


In [54]:
class MeanReductionInverseClassFrequency():
    def __init__(self,k):
        self.k=k
    def __call__(self, x, target):
        sum=0
        classes=target.unique()
        classes=classes[classes!=255]
        for i in classes:
            print(i)
            print(x[target==i].mean())
            print(x[target==i].shape[0])
            sum+=x[target==i].mean()*(x[target!=255].shape[0]/x[target==i].shape[0])**self.k
        return sum/len(classes)

In [57]:
criterion = nn.CrossEntropyLoss(ignore_index=255, reduction='none')
reduction=MeanReductionInverseClassFrequency(0)
l=reduction(criterion(out,iddaTestDiff[0][1].unsqueeze(0)),iddaTestDiff[0][1].unsqueeze(0))
print(l)

tensor(0)
tensor(1.8140, grad_fn=<MeanBackward0>)
770333
tensor(1)
tensor(6.9221, grad_fn=<MeanBackward0>)
27097
tensor(2)
tensor(3.4182, grad_fn=<MeanBackward0>)
118759
tensor(3)
tensor(9.3305, grad_fn=<MeanBackward0>)
28264
tensor(4)
tensor(0.3525, grad_fn=<MeanBackward0>)
598
tensor(5)
tensor(4.3332, grad_fn=<MeanBackward0>)
16253
tensor(6)
tensor(3.3715, grad_fn=<MeanBackward0>)
9220
tensor(8)
tensor(0.7109, grad_fn=<MeanBackward0>)
319988
tensor(10)
tensor(7.6629, grad_fn=<MeanBackward0>)
582134
tensor(11)
tensor(7.3871, grad_fn=<MeanBackward0>)
195
tensor(13)
tensor(4.9312, grad_fn=<MeanBackward0>)
188342
tensor(4.5667, grad_fn=<DivBackward0>)
